# Translate image to corresponding alphabhet using pre-trained model

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.preprocessing import image
import numpy as np
import pandas as pd
from pathlib import Path
import imutils
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
%matplotlib inline




Using TensorFlow backend.


In [2]:
IMAGE_SIZE = 50 #We'll be workign with 50 * 50 pixel images
LABELS = [chr(c) for c in range(ord('A'), ord('Z') + 1)]
LABELS.append("nothing")
LABELS.append("space")

MODEL_PATH = "trained_model\my_model.h5"

In [3]:
from keras.models import load_model
model = load_model(MODEL_PATH)

In [ ]:
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

#  Visualisation of CNN Model


<img src = "model_plot.png">

In [4]:
IMG_PATH = "test.jpg"

def preprocess_image(IMG_PATH):
    """returns image array by preprocessing the image
    Keyword arguments:
    IMG_PATH: path of the image
    Example: img_array = preprocess_image("a.jpg")
    """
    try:
        #Load image with target size and convert img to array
        img = image.load_img(IMG_PATH, target_size = (IMAGE_SIZE,IMAGE_SIZE))
        img_array = image.img_to_array(img)
       # imutils.resize(img,IMAGE_SIZE,IMAGE_SIZE)
        #Change color sapce to gray
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)

        #Reshape array to l * w * channels
        img_array = img_array.reshape(IMAGE_SIZE, IMAGE_SIZE, 1)
        
        #Normalize th array
        img_array /= 225.0
        
        #Expand Dimension of the array as our model expects a 4D array
        img_array = np.expand_dims(img_array, axis = 0)
        return img_array
        
    except Exception as e:
        print(e)
        return None
    

def which_letter(IMG_PATH):
    """returns predicted label using the model or None if exception occurs
    Keyword arguments:
    IMG_PATH: path of the image
    eg:
        print(which_letter("sample.jpeg"))
    """
    try:
        img_array = preprocess_image(IMG_PATH)
        preds = model.predict(img_array)
        most_likely_class_index = int(np.argmax(preds))
        return LABELS[most_likely_class_index]
    except Exception as e:
        print(e)
        return None

result = which_letter(IMG_PATH)
if result:
    print(result)


Z


# Get Frames from Webcam and translate

In [5]:
def getMaxContour(contours, minArea = -1):
    maxC = None
    maxArea = minArea
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if(area > maxArea):
            maxArea = area
            maxC = cnt
    return maxC

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    cv2.rectangle(frame,(0, 0),(320, 480),(255,0,0),3) # bounding box which captures ASL sign to be detected by the system
    img1 = frame[0: 480, 0: 320]
    img_ycrcb = cv2.cvtColor(img1, cv2.COLOR_BGR2YCR_CB)
    blur = cv2.GaussianBlur(img_ycrcb ,(11,11),0)
    skin_ycrcb_min = np.array((0, 138, 67))
    skin_ycrcb_max = np.array((255, 173, 133))
    mask = cv2.inRange(blur, skin_ycrcb_min, skin_ycrcb_max)  # detecting the hand in the bounding box using skin detection
    _,contours, _ = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL, 2) 
    cnt= getMaxContour(contours, minArea = 2000)
    x, y, w, h = cv2.boundingRect(cnt)
    if w > 0 and h > 0:
        mx = max(w, h, 200)
    else:
        mx = 0
    cv2.rectangle(img1,(x - mx // 3,y),(x + 2 * mx // 3,y + mx),(0,255,0),2)
    cv2.imshow("frame",frame)
    cv2.imshow('img1', img1)
    cv2.imshow("mask", mask)
   
    hand = img1[y: y + mx, x - mx // 3: x + 2 * mx // 3]

        
    c = cv2.waitKey(1) & 0xff
    if c == 32:
         cv2.imwrite("test.jpg", hand)
    if c == 27:
        break
cap.release()
cv2.destroyAllWindows()